In [1]:
# !pip install --upgrade gdown

# # Main link
# !gdown --id '1N1eVIDe9hKM5uiNRGmifBlwSDGiVXPJe' --output libriphone.zip
# # !gdown --id '1qzCRnywKh30mTbWUEjXuNT2isOCAPdO1' --output libriphone.zip

# !unzip -q libriphone.zip
# !ls libriphone

In [2]:
import numpy as np
import torch
import random

def same_seeds(seed):
    random.seed(seed) 
    np.random.seed(seed)  
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [3]:
print("pytorch version:",torch.__version__)
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
else: 
    device = 'cpu'
print("use",device,"now!")

pytorch version: 1.13.1
use cuda now!


In [4]:
import os
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, random_seed=1213):
    class_num = 41 # NOTE: pre-computed, should not need change

    if split == 'train' or split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        for line in open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]
        
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(random_seed)
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

    elif mode == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode == 'train':
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode == 'train':
          label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode == 'train':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode == 'train':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode == 'train':
      print(y.shape)
      return X, y
    else:
      return X


# Dataset

In [5]:
import torch
from torch.utils.data import Dataset

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


# Model
Feel free to modify the structure of the model.

In [6]:
# import torch.nn as nn

# class Classifier(nn.Module):
#     def __init__(self):
#         super(Classifier, self).__init__()
#         self.lstm = nn.GRU(39,1024,5,batch_first = True)
        
#         self.classifier = nn.Sequential(
#             nn.Linear(17*1024,2048),
#             nn.BatchNorm1d(2048),
#             nn.ReLU(),
#             nn.Dropout(0.2),
# #             nn.Linear(2048,41),
#             nn.Linear(2048,1024),
#             nn.BatchNorm1d(1024),
#             nn.ReLU(),
#             nn.Dropout(0.2),
#             nn.Linear(1024,41),
#         )
        
#         self.criterion = nn.CrossEntropyLoss() 
        
#     def forward(self, x):
#         x = x.view(-1,17,39)
#         x,_ = self.lstm(x)
#         x = x.contiguous().view(x.size(0),-1)
#         x = self.classifier(x)
#         return x

In [7]:
# import torch.nn as nn
# import torch.nn.functional as F

# class Classifier(nn.Module):
#     def __init__(self, batch_size, num_layers=3, hidden_dim=356, seq_length=concat_nframes):  #inputsize: batch_size * concat_nframes * 39
#         super(Classifier, self).__init__()
#         self.batch_size = batch_size
#         self.num_layers = num_layers
#         self.hidden_dim = hidden_dim
#         self.seq_length = seq_length
#         self.lstm = nn.LSTM(input_size=39, hidden_size=hidden_dim, num_layers= num_layers, batch_first=True, dropout=0.4, bidirectional=True)
#         # self.h0 = torch.zeros(num_layers * 2, batch_size, hidden_dim).to(device)
#         # self.c0 = torch.zeros(num_layers * 2, batch_size, hidden_dim).to(device)
#         self.fc = nn.Sequential(
#             nn.LeakyReLU(0.1),
#             nn.BatchNorm1d(2 * hidden_dim),
#             nn.Dropout(0.4),
#             nn.Linear(2 * hidden_dim, hidden_dim),
#             nn.LeakyReLU(0.1),
#             nn.BatchNorm1d(hidden_dim),
#             nn.Dropout(0.4),
#             nn.Linear(hidden_dim, 41)
#         )

#     def forward(self, x):
#         x = x.view(-1,17,39)
#         x, _ = self.lstm(x)
#         x = x[:,self.seq_length//2]
#         x = self.fc(x)
#         return x
     

In [8]:
# data prarameters
concat_nframes = 55              # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.99               # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 24                        # random seed
batch_size = 2048                # batch size
num_epoch = 30                  # the number of training epoch
learning_rate = 1e-5         # learning rate
model_path = './model.ckpt'     # the path where the checkpoint will be saved

In [34]:

import torch
import torch.nn as nn
import torch.nn.functional as F



class Classifier(nn.Module):
    def __init__(self, batch_size, num_layers=3, hidden_dim=512, seq_length=concat_nframes):  #inputsize: batch_size * concat_nframes * 39
        super(Classifier, self).__init__()
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.seq_length = seq_length
        self.lstm = nn.LSTM(input_size=39, hidden_size=hidden_dim, num_layers= num_layers, batch_first=True, dropout=0.4, bidirectional=True)
        self.fc = nn.Sequential(
            nn.LeakyReLU(0.1),
#             nn.ReLU(),
            nn.BatchNorm1d(2 * hidden_dim),
            nn.Dropout(0.3),
            nn.Linear(2 * hidden_dim, hidden_dim),
            nn.LeakyReLU(0.1),
#             nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, 41)
        )

    def forward(self, x):
        x = x.view(-1,self.seq_length,39)
        x, _ = self.lstm(x)
        x = x[:,self.seq_length//2]
        x = self.fc(x)
        return x
     

# Hyper-parameters

In [35]:
# # data prarameters
# # TODO: change the value of "concat_nframes" for medium baseline
# concat_nframes = 17   # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
# train_ratio = 0.99   # the ratio of data used for training, the rest will be used for validation

# # training parameters
# seed = 24          # random seed
# batch_size = 2048        # batch size
# num_epoch = 10        # the number of training epoch
# learning_rate = 1e-4      # learning rate
# model_path = './model.ckpt'  # the path where the checkpoint will be saved

# # model parameters
# # TODO: change the value of "hidden_layers" or "hidden_dim" for medium baseline
# input_dim = 39 * concat_nframes  # the input dim of the model, you should not change the value
# hidden_layers = 12          # the number of hidden layers
# hidden_dim = 1024           # the hidden dim
# output_classes = 41
# input_size = 39 * concat_nframes  # the input dim of the model, you should not change the value
# hidden_size = 64          # the number of hidden layers
# num_layers = 3           # the hidden dim

# Dataloader

In [23]:
from torch.utils.data import DataLoader
import gc

same_seeds(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

# preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

DEVICE: cuda
[Dataset] - # phone classes: 41, number of utterances for train: 3394


3394it [00:20, 163.14it/s]


[INFO] train set
torch.Size([2094906, 2145])
torch.Size([2094906])
[Dataset] - # phone classes: 41, number of utterances for val: 35


35it [00:00, 40.43it/s]


[INFO] val set
torch.Size([21888, 2145])
torch.Size([21888])


# Training

In [33]:
# create model, define a loss function, and optimizer
# model = Classifier(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers).to(device)
model = Classifier(batch_size).to(device)
# model = Classifier(batch_size).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate  * 200)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=2, T_mult=4, eta_min=0)  
     
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    print("第%d個epoch的learning rate：%f" % (epoch+1, optimizer.param_groups[0]['lr']))
    
    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)
        outputs = model(features) 
        
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward() 
        optimizer.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()
    scheduler.step()
    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                features, labels = batch
                features = features.to(device)
                labels = labels.to(device)
                
                outputs = model(features)
                
                loss = criterion(outputs, labels) 
                
                _, val_pred = torch.max(outputs, 1) 
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')

第1個epoch的learning rate：0.002000


  0%|          | 0/1023 [00:00<?, ?it/s]


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [2048, 55, 1024]], which is output 0 of CudnnRnnBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [63]:
# # create model, define a loss function, and optimizer
# # model = Classifier(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers).to(device)
# model = Classifier(batch_size).to(device)
# model = Classifier(batch_size).to(device)
# criterion = nn.CrossEntropyLoss() 
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate  * 200)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 
#                                         T_0=2, T_mult=4, eta_min=0)  
     
# # model = RNN_GRU(input_dim, hidden_dim, hidden_layers, output_classes).to(device)


# best_acc = 0.0
# for epoch in range(num_epoch):
#     train_acc = 0.0
#     train_loss = 0.0
#     val_acc = 0.0
#     val_loss = 0.0
    
#     # training
#     model.train() # set the model to training mode
#     for i, batch in enumerate(tqdm(train_loader)):
#         features, labels = batch
#         features = features.to(device)
#         labels = labels.to(device)
        
#         optimizer.zero_grad() 
#         outputs = model(features) 
        
#         loss = criterion(outputs, labels)
#         loss.backward() 
#         optimizer.step() 
        
#         _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
#         train_acc += (train_pred.detach() == labels.detach()).sum().item()
#         train_loss += loss.item()
    
#     # validation
#     model.eval() # set the model to evaluation mode
#     with torch.no_grad():
#         for i, batch in enumerate(tqdm(val_loader)):
#             features, labels = batch
#             features = features.to(device)
#             labels = labels.to(device)
#             outputs = model(features)
            
#             loss = criterion(outputs, labels) 
            
#             _, val_pred = torch.max(outputs, 1) 
#             val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
#             val_loss += loss.item()

#     print(f'[{epoch+1:03d}/{num_epoch:03d}] Train Acc: {train_acc/len(train_set):3.5f} Loss: {train_loss/len(train_loader):3.5f} | Val Acc: {val_acc/len(val_set):3.5f} loss: {val_loss/len(val_loader):3.5f}')

#     # if the model improves, save a checkpoint at this epoch
#     if val_acc > best_acc:
#         best_acc = val_acc
#         torch.save(model.state_dict(), model_path)
#         print(f'saving model with acc {best_acc/len(val_set):.5f}')


In [64]:
del train_set, val_set
del train_loader, val_loader
gc.collect()

479

# Testing
Create a testing dataset, and load model from the saved checkpoint.

In [65]:
# load data
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 857


857it [00:10, 80.05it/s]


[INFO] test set
torch.Size([527364, 2145])


In [66]:
# load model
# model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
model = Classifier(batch_size).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [67]:
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
#         features = features.to(device)
        features = features.to(device)
        outputs = model(features)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 258/258 [00:35<00:00,  7.30it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [68]:
with open('prediction_rnn_test55_hiddendim_512_seed24_dropout0.3_gelu.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))